In [2]:
##Importamos librerias 

import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy import stats 
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from src import soporte_funciones as spt
# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('files/df_final.csv')
df.head(1)

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,salary
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,92552.0


In [4]:
## Filtramos el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'

df_filtrado = df[['flights_booked', 'education']]
df_filtrado.head()

,flights_booked,education
0,3,Bachelor
1,10,College
2,6,College
3,0,Bachelor
4,0,Bachelor


#### Análisis descriptivo: 

In [5]:
#Promedio de vuelos reservados por nivel educativo
df.groupby('education')['flights_booked'].mean()

education
Bachelor                4.110288
College                 4.169744
Doctor                  4.175512
High School or Below    4.176209
Master                  4.200700
Name: flights_booked, dtype: float64

In [6]:
#Conteo de vuelos reservados por nivel educativo
df_filtrado.groupby("education")["flights_booked"].count()

education
Bachelor                252567
College                 102260
Doctor                   17731
High School or Below     18915
Master                   12287
Name: flights_booked, dtype: int64

In [7]:
#Descripción de estadísticas generales: 
df_filtrado.groupby("education")["flights_booked"].describe().T


education,Bachelor,College,Doctor,High School or Below,Master
count,252567.000000,102260.000000,17731.000000,18915.000000,12287.000000
mean,4.110288,4.169744,4.175512,4.176209,4.200700
std,5.221671,5.246040,5.256971,5.239267,5.213956
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,8.000000,8.000000,8.000000,8.000000,8.000000
max,21.000000,21.000000,21.000000,21.000000,21.000000


In [8]:
#Analisis: 
#Count: la mayor cantidad de registros es de Bachelor y la menor es Master
#Mean: el promedio de vuelos es bastante similar 
#Desviación estandar: también es consistente (no se alejan tanto del mean)
#Tanto min, median, 3er cuartil y max son consistentes.  

In [9]:
## Verificamos la moda de vuelos reservados por grupo: 

moda_por_educacion = df_filtrado.groupby('education')['flights_booked'].apply(pd.Series.mode).reset_index()

In [10]:
moda_por_educacion

,education,level_1,flights_booked
0,Bachelor,0,0
1,College,0,0
2,Doctor,0,0
3,High School or Below,0,0
4,Master,0,0


In [11]:
# Analizamos porque es 0

In [12]:
df_filtrado.shape

(403760, 2)

In [13]:
df_sin_vuelos = df_filtrado[df_filtrado['flights_booked'] == 0]
df_sin_vuelos

,flights_booked,education
3,0,Bachelor
4,0,Bachelor
5,0,Bachelor
6,0,Bachelor
7,0,Bachelor
...,...,...
403754,0,Bachelor
403755,0,College
403756,0,Doctor
403758,0,College


In [14]:
df_con_vuelos = df_filtrado[df_filtrado['flights_booked'] > 0]
df_con_vuelos

,flights_booked,education
0,3,Bachelor
1,10,College
2,6,College
8,6,Bachelor
10,3,Bachelor
...,...,...
403748,15,Doctor
403749,14,College
403750,11,Bachelor
403752,1,College


In [15]:
## Conclusión: casi la mitad de los registros es 0, por lo que la moda nos devuelve ese número. 
## Filtramos los ceros para ver si cambia la moda: 

In [16]:

# Recalcular la moda 
moda_por_educacion_vuelos = df_con_vuelos.groupby('education')['flights_booked'].apply(pd.Series.mode).reset_index()
moda_por_educacion_vuelos

,education,level_1,flights_booked
0,Bachelor,0,3
1,College,0,3
2,Doctor,0,3
3,High School or Below,0,3
4,Master,0,3


In [17]:
df_con_vuelos.groupby("education")["flights_booked"].describe().T


education,Bachelor,College,Doctor,High School or Below,Master
count,129303.000000,52834.000000,9193.000000,9855.000000,6447.000000
mean,8.028607,8.070523,8.053519,8.015525,8.005894
std,4.669000,4.667465,4.697898,4.680893,4.620198
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000,4.000000,4.000000
50%,8.000000,8.000000,8.000000,8.000000,8.000000
75%,11.000000,11.000000,11.000000,11.000000,11.000000
max,21.000000,21.000000,21.000000,21.000000,21.000000


### Pruebas estadísticas: 

In [18]:
## Orden educativo en Canadá:
#High School or Below
#College (programas técnicos o diplomas)
#Bachelor (grado universitario)
#Master (posgrado)
#Doctor (PhD) (doctorado)

In [19]:
##Analizamos si hay diferencia significativa en el número de vuelos reservados entre los grupos universitario y no universitario.

In [20]:
## 1) Utilizando el DF entero (que incluye 0 en la columna de vuelos reservados)

In [21]:
## Separamos grupos según orden educativo, para obtener 2 grupos: 

grupo_no_universitario = df_filtrado[df_filtrado['education'].isin(['High School or Below', 'College'])]['flights_booked']
grupo_universitario = df_filtrado[df_filtrado['education'].isin(['Bachelor', 'Master', 'Doctor'])]['flights_booked']

In [22]:
## Hacemos crosstable 

pd.crosstab(df_filtrado['education'], df_filtrado['flights_booked'])

flights_booked,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
education,,,,,,,,,,,,,,,,,,,,,
Bachelor,123264,7949,8423,11436,8033,9444,8606,9050,8958,8790,...,6492,6789,4358,4081,2678,2392,1813,962,708,347
College,49426,3115,3508,4612,3209,3881,3505,3687,3613,3614,...,2677,2698,1814,1696,1145,1023,766,354,308,121
Doctor,8538,579,589,818,586,605,608,662,665,614,...,426,464,326,286,207,158,142,79,52,25
High School or Below,9060,634,715,794,580,722,636,701,686,693,...,485,499,336,330,186,194,139,65,65,18
Master,5840,390,406,568,413,432,492,461,435,453,...,302,332,223,189,138,132,98,36,23,17


In [23]:
## La tabla de contingencia no nos da información valiosa. Entonces: es un problema de medias. 

In [24]:
## Verificamos normalidad de los datos: 

spt.normalidad(df_filtrado, 'flights_booked')

Para la columna flights_booked los datos no siguen una distribución normal.


/home/ariana91/anaconda4/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 403760.
  res = hypotest_fun_out(*samples, **kwds)


In [25]:
## Los datos no son normales, por lo que hacemos el test MW: 

In [26]:
mannwhitneyu(grupo_no_universitario, grupo_universitario)

MannwhitneyuResult(statistic=17216065553.0, pvalue=0.002942081262120518)

In [27]:
## definimos P value
_, p_value = mannwhitneyu(grupo_no_universitario,grupo_universitario)

In [28]:
p_value < 0.05

True

In [29]:
## Conclusión: el p_value es true, por lo que hay una diferencia significativa entre los grupos. Al separar los niveles educativos por 'Universitario' vs 'No universitarios', incluyendo el total de vuelos reservados tanto 
## como los registros que no reservaron ningún vuelo, el análisis indica que si existe una diferencia significativa entre los vuelos reservados 
## por ambos grupos. Es decir que el número de vuelos reservados no se distribuye de manera uniforme entre los grupos.

In [30]:
## 2) Utilizando el DF que solo tiene valores mayores a 0 en la columna de vuelos reservados:

In [31]:
## Separamos grupos según orden educativo, para obtener 2 grupos: 

grupo_no_universitario_1 = df_con_vuelos[df_con_vuelos['education'].isin(['High School or Below', 'College'])]['flights_booked']
grupo_universitario_1 = df_con_vuelos[df_con_vuelos['education'].isin(['Bachelor', 'Master', 'Doctor'])]['flights_booked']

In [32]:
## Hacemos MW: 
mannwhitneyu(grupo_no_universitario_1, grupo_universitario_1)

MannwhitneyuResult(statistic=4562361209.0, pvalue=0.12501068282706929)

In [33]:
## definimos P value
_, p_value_1 = mannwhitneyu(grupo_no_universitario_1,grupo_universitario_1)

In [34]:
p_value_1< 0.05

False

In [ ]:
## Conclusión: el p_value es false, por lo que no hay una diferencia significativa entre los grupos. Al separar los niveles educativos por 'Universitario' vs 'No universitarios', excluyendo los casos en los que los registros de vuelos reservados eran 0,
# el análisis indica que no existe una diferencia significativa entre los vuelos reservados por ambos grupos. 
# Es decir que el número de vuelos efectivamente reservados se distribuye de manera uniforme entre los grupos.